## **Surrogate modelling to design a digital twin for ground source heat exchangers**

Author: Emilio Osuna

Data exploration is done in `data_exploration.ipynb`

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

The purpose of this notebook is to perform the simulations of a surrogate model based on [Yildiz & Stirling, 2022](https://doi.org/10.1016/j.geothermics.2022.102351), where a finite difference model is used to predict soil temperature.

The governing PDE model is a transient heat conduction equation:

\begin{equation}
    \frac{\partial T}{\partial t} = \alpha \frac{\partial^2 T}{\partial z^2} + \frac{\partial \alpha}{\partial z} \frac{\partial T}{\partial z}
\end{equation}


The thermal diffusivity $\alpha$ changes over time and space. The upper temperature measured at the top of the lysimeter is consider as the boundary condition (Dirichlet). The heat flux at the lower boundary is calculated by adding the measured heat flux at 940 mm depth, and the energy stored in the soil layer between 850 mm and 940 mm:

\begin{equation}
    \Delta S = \int_{z=850}^{z=940} C \frac{\partial T}{\partial t} \, dz + \int_{z=850}^{z=940} T \frac{\partial C}{\partial t} \, dz
\end{equation}

As the hydrological regime of the soil is fluctuating throughout the field-testing period, a relationship between the volumetric water content ($\theta$) and the thermal property in question is needed:

\begin{equation}
\alpha_{\text{sand}} = 0.25 + \frac{0.64}{1 + e^{-1.72(\theta - 6.01)}}
\end{equation}

\begin{equation}
\alpha_{\text{topsoil}} = 0.23 + \frac{0.25}{1 + e^{-0.78(\theta - 11.3)}}
\end{equation}

\begin{equation}
C_{\text{sand}} = 296.4 + \frac{52.7 + 8.32(\theta - 5.27)}{1 + e^{-3.24(\theta - 5.27)}}
\end{equation}

## PINN Model

In [2]:
class PINN(nn.Module):
    def __init__(self, num_hidden_layers=4, num_neurons=64):
        super(PINN, self).__init__()
        
        # Input layer: (t, z, theta) -> hidden
        self.input_layer = nn.Linear(3, num_neurons)
        
        # Hidden layers
        self.hidden_layers = nn.ModuleList(
            [nn.Linear(num_neurons, num_neurons) for _ in range(num_hidden_layers)]
        )
        
        # Output layer: hidden -> T
        self.output_layer = nn.Linear(num_neurons, 1)
        
        # Activation
        self.activation = nn.Tanh()
        
        # Initialize weights
        self._init_weights()

    def _init_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.xavier_normal_(m.weight)
                nn.init.zeros_(m.bias)

    def forward(self, t, z, theta):
        # Concatenate inputs
        x = torch.cat([t, z, theta], dim=1)  # shape: (batch_size, 3)
        
        # Forward pass
        x = self.input_layer(x)
        x = self.activation(x)
        for hl in self.hidden_layers:
            x = hl(x)
            x = self.activation(x)
        x = self.output_layer(x)
        
        return x  # shape: (batch_size, 1)